In [33]:
# %%capture
# !pip install langchain==0.1.0 chromadb tiktoken openai

In [34]:
# %%capture
# !pip install langchain_experimental==0.0.49

In [35]:
# %%capture
# # !pip install pypdf

In [36]:
# %%capture
# !pip install -r r.txt

In [1]:
!du -h ./demo_db/

du: cannot access './demo_db/': No such file or directory


In [2]:
!rm -rf ./demo_db

In [3]:
import os
from io import StringIO
import pandas as pd
import base64
from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)
from dotenv import dotenv_values

In [4]:
config = dotenv_values(".env")

In [5]:
openai_api_key = config['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = openai_api_key

In [6]:
llm = ChatOpenAI(temperature=0.25, verbose=True,model_name= 'gpt-3.5-turbo-0301')
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
loader = PyPDFLoader("./RTU.pdf")
# # loader = PyPDFLoader("/home/jueli/Downloads/[Kernighan-Ritchie]The_C_Programming_Language.pdf")
# loader = PyPDFLoader("/home/jueli/Downloads/Cognitive Neuroscience of Language (David Kemmerer) (Z-Library).pdf")
# loader = PyPDFLoader("/home/jueli/Downloads/Tutorial_EDIT.pdf")
pages=loader.load()
text_splitter = RecursiveCharacterTextSplitter (chunk_size=200, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

In [8]:
len(texts)

679

In [9]:
PERSISTENT_DIR = 'demo_db'

In [10]:
store = Chroma.from_documents(texts, embeddings, collection_name='Pdf', persist_directory=PERSISTENT_DIR)
vectorstore_info = VectorStoreInfo(
    name="Pdf",
    description=" A pdf file to answer your questions",
    vectorstore=store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

pdfAgent = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [11]:
response = pdfAgent.run("what is RTrU used for?")

/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I don't know what RTrU is, so I need to find out more information.
Action: Pdf
Action Input: "RTrU filetype:pdf"
Observation:  I don't know.
Thought:I need to try a different search query to find information about RTrU.
Action: Pdf
Action Input: "RTrU technology filetype:pdf"
Observation:  I don't know.
Thought:I need to try a different approach to find information about RTrU.
Action: Pdf_with_sources
Action Input: "What is RTrU and what is it used for?"
Observation: {"answer": " RTrU is a device used for logging and sending data when network or server connection is restored. It can be configured through a webpage or vice versa.\n", "sources": "./RTU.pdf"}
Thought:I have found the answer to the question about RTrU and have provided sources, so I can provide the final answer.
Final Answer: RTrU is a device used for logging and sending data when network or server connection is restored. It can be configured through a webpage or vice versa.

> Finis

In [12]:
print(response)

RTrU is a device used for logging and sending data when network or server connection is restored. It can be configured through a webpage or vice versa.


In [13]:
df = pd.read_csv('./train.csv')

In [14]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [15]:
pandasAgent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True)

In [16]:
pandasAgent.run("What is the average Age?")



> Entering new AgentExecutor chain...
Thought: I need to use the `mean()` function on the `Age` column of the dataframe.
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought:I now know the average age.
Final Answer: The average age is 29.69911764705882.

> Finished chain.


'The average age is 29.69911764705882.'

In [17]:
pandasAgent.run("What is the average Age per parch?")




> Entering new AgentExecutor chain...
Thought: We need to group the data by Parch and then calculate the mean of the Age column for each group.
Action: python_repl_ast
Action Input:
```
df.groupby('Parch')['Age'].mean()
```
Observation: Parch
0    32.178503
1    24.422000
2    17.216912
3    33.200000
4    44.500000
5    39.200000
6    43.000000
Name: Age, dtype: float64
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `We have the average Age per Parch.`

In [18]:
pandasAgent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, handle_parsing_errors=True)

In [19]:
pandasAgent.run("What is the average Age per parch?")




> Entering new AgentExecutor chain...
Thought: We need to group the data by Parch and then calculate the mean of Age for each group.
Action: Use groupby() and mean() functions.
Action Input: `df.groupby('Parch')['Age'].mean()`
Observation: Use groupby() and mean() functions. is not a valid tool, try one of [python_repl_ast].
Thought:Let's use python_repl_ast to execute the command.
Action: python_repl_ast
Action Input: `df.groupby('Parch')['Age'].mean()`
Observation: Parch
0    32.178503
1    24.422000
2    17.216912
3    33.200000
4    44.500000
5    39.200000
6    43.000000
Name: Age, dtype: float64
Thought:The average Age per parch is given by the output above.
Final Answer: The average Age per parch is:
Parch
0    32.178503
1    24.422000
2    17.216912
3    33.200000
4    44.500000
5    39.200000
6    43.000000
Name: Age, dtype: float64

> Finished chain.


'The average Age per parch is:\nParch\n0    32.178503\n1    24.422000\n2    17.216912\n3    33.200000\n4    44.500000\n5    39.200000\n6    43.000000\nName: Age, dtype: float64'